<a href="https://colab.research.google.com/github/svarogjk/shop_revenue_prediction/blob/master/gstore_competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports and Readings

In [0]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
current_palette = sns.color_palette()
# from tqdm import tqdm_notebook
%matplotlib inline 
import io
from google.colab import files

In [0]:
import json
from pandas.io.json import json_normalize
import random

In [0]:
from itertools import product
from sklearn.preprocessing import LabelEncoder

In [0]:
# !pip install fbprophet
# from fbprophet import Prophet

In [0]:
from xgboost import XGBRegressor
from xgboost import plot_importance

In [0]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import r2_score, make_scorer

In [0]:
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [8]:
file_list = drive.ListFile({'q': "'19ry360g_dRMf_i9Gi4A2WrOzE682GA7O' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: gstore_competition.ipynb, id: 1gRRqwzC-VDFe_fOQW9Kc-5YX3Gmofxnt
title: train.csv, id: 1-9OX7E7uSovNTI4-h43oMQpZavJai6sb
title: test.csv, id: 1SohvLEveFD4yhhR_alpJtfgtY5QIYMV4
title: sample_submission.csv, id: 1WLB9e5uswsrR8NWq4yLwFKlXE9uYUZKB
title: all.zip (Unzipped Files), id: 13XaW-vTIJKNDKMrbhamaduANnnPFojkQ
title: all.zip, id: 1Mi8GWSji2h1FGzWkQHYXmRuQNqHcIUlj


In [0]:
train = drive.CreateFile({'id': '1-9OX7E7uSovNTI4-h43oMQpZavJai6sb'})
train.GetContentFile('train.csv')

test_data = drive.CreateFile({'id': '1SohvLEveFD4yhhR_alpJtfgtY5QIYMV4'})
test_data.GetContentFile('test.csv')

submission = drive.CreateFile({'id': '1WLB9e5uswsrR8NWq4yLwFKlXE9uYUZKB'})
test_data.GetContentFile('sample_submission.csv')

### Here we read our DataFrame and transform its json columns into normal ones

In [0]:
def json_read(df):
    
    columns = ['device', 'geoNetwork', 'totals', 'trafficSource']
    p = 0.07
    
    data_frame = df
    
    #Importing the dataset
    df = pd.read_csv(data_frame, 
                     converters={column: json.loads for column in columns}, # loading the json columns properly
                     dtype={'fullVisitorId': 'str'},
                    skiprows=lambda i: i>0 and random.random() > p
                    ) # transforming this column to string
    
    for column in columns: #loop to finally transform the columns in data frame
        #It will normalize and set the json to a table
        column_as_df = json_normalize(df[column]) 
        # here will be set the name using the category and subcategory of json columns
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns] 
        # after extracting the values, let drop the original columns
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
        
    # Printing the shape of dataframes that was imported     
    print(f"Loaded {os.path.basename(data_frame)}. Shape: {df.shape}")
    return df # returning the df after importing and transforming

In [0]:
train = pd.read_csv('train.csv', low_memory=False)
test_data = pd.read_csv('test.csv', low_memory=False)
submission = pd.read_csv('sample_submission.csv', low_memory=False)

In [16]:
train.head(2)

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",1131660440785968503,"{""continent"": ""Asia"", ""subContinent"": ""Western...",1131660440785968503_1472830385,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472830385,1,1472830385
1,Organic Search,20160902,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",377306020877927890,"{""continent"": ""Oceania"", ""subContinent"": ""Aust...",377306020877927890_1472880147,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472880147,1,1472880147


In [13]:
test_data.head(2)

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",6167871330617112363,"{""continent"": ""Asia"", ""subContinent"": ""Southea...",6167871330617112363_1508151024,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""4"", ""pageviews"": ""4""}","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508151024,2,1508151024
1,Organic Search,20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",0643697640977915618,"{""continent"": ""Europe"", ""subContinent"": ""South...",0643697640977915618_1508175522,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""5"", ""pageviews"": ""5"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508175522,1,1508175522


In [14]:
submission.head(2)

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",6167871330617112363,"{""continent"": ""Asia"", ""subContinent"": ""Southea...",6167871330617112363_1508151024,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""4"", ""pageviews"": ""4""}","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508151024,2,1508151024
1,Organic Search,20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",0643697640977915618,"{""continent"": ""Europe"", ""subContinent"": ""South...",0643697640977915618_1508175522,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""5"", ""pageviews"": ""5"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508175522,1,1508175522


## Extract target

In [0]:
train_totals = train.totals.apply(json.loads).apply(pd.Series)

In [18]:
train_totals.head(2)

,bounces,hits,newVisits,pageviews,transactionRevenue,visits
0,1,1,1,1,NaN,1
1,1,1,1,1,NaN,1


In [0]:
train = pd.concat([train, train_totals], axis=1)
train.drop(["totals"], axis=1, inplace=True)

In [0]:
train_totals = train.totals.apply(json.loads).apply(pd.Series)
test_totals = test_data.totals.apply(json.loads).apply(pd.Series)
submission_totals = submission.totals.apply(json.loads).apply(pd.Series)

In [0]:
train = pd.concat([train, train_totals], axis=1)
test_data = pd.concat([test_data, test_totals], axis=1)
submission = pd.concat([submission, submission_totals], axis=1)

In [0]:
train.drop(["totals"], axis=1, inplace=True)
test_data.drop(["totals"], axis=1, inplace=True)
submission.drop(["totals"], axis=1, inplace=True)

In [18]:
train.head(2)

,fullVisitorId,date,bounces,hits,newVisits,pageviews,transactionRevenue,visits
0,1131660440785968503,20160902,1,1,1,1,NaN,1
1,377306020877927890,20160902,1,1,1,1,NaN,1


In [0]:
train_rev = train[~train.transactionRevenue.isnull()]

In [20]:
train_rev.head()

,fullVisitorId,date,bounces,hits,newVisits,pageviews,transactionRevenue,visits
752,6194193421514403509,20160902,NaN,11,1,11,37860000,1
753,5327166854580374902,20160902,NaN,11,NaN,10,306670000,1
799,8885051388942907862,20160902,NaN,13,NaN,11,68030000,1
802,0185467632009737931,20160902,NaN,13,NaN,12,26250000,1
859,3244885836845029978,20160902,NaN,17,NaN,14,574150000,1


In [21]:
train_rev.shape

(11515, 8)

In [22]:
train_rev.fullVisitorId.unique().shape

(10366,)

In [23]:
train.fullVisitorId.unique().shape

(742735,)

In [24]:
train.shape

(903653, 8)

In [25]:
len(set(test_data.fullVisitorId.unique()) & set(train_rev.fullVisitorId.unique()))

365

In [29]:
len(set(test_data.fullVisitorId.unique()) & set(submission.fullVisitorId.unique()))

652237

In [30]:
len(set(test_data.fullVisitorId.unique()))

652237

In [26]:
train_rev.date.dtype

dtype('int64')

In [27]:
print(train.date.min(), train.date.max())

20160801 20170801


In [28]:
print(test_data.date.min(), test_data.date.max())

20170802 20180430


In [34]:
submission.head()

,fullVisitorId,date,bounces,hits,newVisits,pageviews,visits
0,6167871330617112363,20171016,NaN,4,NaN,4,1
1,0643697640977915618,20171016,NaN,5,1,5,1
2,6059383810968229466,20171016,NaN,7,1,7,1
3,2376720078563423631,20171016,NaN,8,1,4,1
4,2314544520795440038,20171016,NaN,9,1,4,1


In [32]:
test_data.head()

,fullVisitorId,date,bounces,hits,newVisits,pageviews,visits
0,6167871330617112363,20171016,NaN,4,NaN,4,1
1,0643697640977915618,20171016,NaN,5,1,5,1
2,6059383810968229466,20171016,NaN,7,1,7,1
3,2376720078563423631,20171016,NaN,8,1,4,1
4,2314544520795440038,20171016,NaN,9,1,4,1


In [33]:
print(submission.shape, test_data.shape)

(804684, 7) (804684, 7)


In [0]:
loyal_df = pd.DataFrame()

In [0]:
loyal_df["fullVisitorId"] = list(set(test_data.fullVisitorId.unique()) & set(train_rev.fullVisitorId.unique()))

In [39]:
loyal_df.shape

(365, 1)

In [38]:
loyal_df.head()

,fullVisitorId
0,7803437096506812413
1,1996186587769697436
2,7545000103569508809
3,1048387524675825671
4,2128287929750116366


In [0]:
train_rev = train_rev[["fullVisitorId", "date", "transactionRevenue"]]

In [0]:
test_data = test_data[["fullVisitorId", "date"]]

In [47]:
test_data.shape

(804684, 2)

In [0]:
loyal_df = loyal_df.join(train_rev.set_index("fullVisitorId"), on="fullVisitorId")

In [43]:
loyal_df.head()

,fullVisitorId,date,transactionRevenue
0,7803437096506812413,20170725,870150000
1,1996186587769697436,20170801,43640000
2,7545000103569508809,20170528,14350000
2,7545000103569508809,20170528,42700000
2,7545000103569508809,20170603,7980000


In [44]:
loyal_df.shape

(535, 3)

In [46]:
loyal_df.fullVisitorId.unique().shape

(365,)